Generate an isoscape from a variational inferenence model

In [1]:
MODEL_SAVE_LOCATION = "/content/gdrive/MyDrive/amazon_rainforest_files/variational/model/random_boosted_isorix_carbon_ensemble.tf" #@param
TRANSFORMER_SAVE_LOCATION = "/content/gdrive/MyDrive/amazon_rainforest_files/variational/model/random_boosted_isorix_carbon_ensemble.pkl" #@param
OUTPUT_RASTER_NAME = "amazon_test_old_res" #@param
USE_LOCAL_DRIVE = False #@param {type:"boolean"}

# Number of pixels on each dimension
RESOLUTION_X = 300 #@param
RESOLUTION_Y = 300 #@param
AMAZON_ONLY = False #@param {type: "boolean"}

In [2]:
#@title Imports and modules.

import tensorflow as tf
import joblib
import os
import sys
from matplotlib import rc

rc('animation', html='jshtml')

!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
ddfimport.ddf_import_common()

Cloning into 'ddf_common_stub'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14 (delta 5), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (14/14), 6.14 KiB | 6.14 MiB/s, done.
Resolving deltas: 100% (5/5), done.
executing checkout_branch ...
b''
main branch checked out as readonly. You may now use ddf_common imports


In [3]:
import model
import raster

# Access data stored on Google Drive if not reading data locally.
if not USE_LOCAL_DRIVE:
  raster.mount_gdrive()

Mounted at /content/gdrive


# Import Tensorflow model and scalers

In [4]:
vi_model = model.TFModel(MODEL_SAVE_LOCATION, TRANSFORMER_SAVE_LOCATION)

In [ ]:
raster.generate_isoscapes_from_variational_model(
  vi_model, RESOLUTION_X, RESOLUTION_Y, OUTPUT_RASTER_NAME, amazon_only=AMAZON_ONLY)

Driver: GTiff/GeoTIFF
Size is 941 x 937 x 12
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0000000000241, 5.29166666665704)
Pixel Size = (0.04166666666665718, -0.04166666666667143)
Driver: GTiff/GeoTIFF
Size is 941 x 937 x 12
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0000000000241, 5.29166666665704)
Pixel Size = (0.04166666666665718, -0.04166666666667143)
Driver: GTiff/GeoTIFF
Size is 942 x 936 x 1
Projection is GEOGCS["SIRGAS 2000",DATUM["Sistema_de_Referencia_Geocentric

# Rendering

## Means

In [ ]:
means_anim = raster.animate(raster.load_raster(raster.get_raster_path(OUTPUT_RASTER_NAME+".tiff"), use_only_band_index=0), 1, 1)
means_anim

# Vars

In [ ]:
vars_anim = raster.animate(raster.load_raster(raster.get_raster_path(OUTPUT_RASTER_NAME+".tiff"), use_only_band_index=1), 1, 1)
vars_anim